In [ ]:
using DifferentialEquations
using ODEInterfaceDiffEq
using Plots
using Plots.PlotMeasures

# Heat equation

We use a discretization through the method of lines (MOL) of the heat equation :

$$
\left\{
\begin{aligned}
&\partial_t  u(t,x) = \partial^2_{xx} u(t,x) \qquad{} t\geq t_0 \\
&u(t_0,x) = u_0(x)
\end{aligned}
\right.
$$

In a first part, we focus on this equation on a bounded domain, with homogeneous Neuman boundary conditions.

In [ ]:
function heat1d!(du, u, p, t)
  nx, dx = p
  nx = convert(UInt64, nx)
    
  oneoverdxdx = 1.0/(dx*dx)
  du[1] = oneoverdxdx * (-2*u[1] + 2*u[2])
  du[2:nx-1] = oneoverdxdx * (u[1:nx-2] - 2*u[2:nx-1] + u[3:nx])
  du[nx] = oneoverdxdx * (2*u[nx-1] - 2*u[nx])
end;

## Integration of the dynamics for a given discretization

### Dopri5 integration

In [ ]:
xmin = -2.0
xmax =  2.0
nx = 401

tini = 0. 
tend = 0.1      
tspan = (tini, tend)

x = collect(range(xmin, xmax, length=nx))
freq = pi/((xmax-xmin)/2)
uini = cos.(freq*x)

uexa = exp.(-(freq^2)*(tend-tini))*uini

dx = (xmax-xmin)/(nx-1)

p = [nx, dx]

tol = 1e-6
prob = ODEProblem(heat1d!, uini, tspan, p)
sol = solve(prob, Tsit5(), abstol=tol, reltol=tol)
println("Number of function evaluations : ", sol.destats.nf)
println("Number of accepted steps :       ", sol.destats.naccept)
println("Number of rejected steps :       ", sol.destats.nreject)

##println("Final time : ", sol.t[end])

err = abs.(sol[end] - uexa)

ps = plot(sol[end], size=(900,300), leg=false, xaxis = ("x"), margin=5mm, 
          title="Heat solution", titlefontsize=12, markershape=:xcross, markersize=2)
pe = plot(x, err, xaxis = ("x"), leg=false, size=(900,600), margin=5mm, 
          title="Error", titlefontsize=12, markershape=:xcross, markersize=2)
plot(ps, pe, layout=(2,1))

### Rock 4 integration

In [ ]:
xmin = -2.0
xmax =  2.0
nx = 400

tini = 0. 
tend = 0.1      
tspan = (tini, tend)

x = collect(range(xmin, xmax, length=nx))
freq = pi/((xmax-xmin)/2)
uini = cos.(freq*x)

uexa = exp.(-(freq^2)*(tend-tini))*uini

dx = (xmax-xmin)/(nx-1)

p = [nx, dx]

tol = 1e-6
prob = ODEProblem(heat1d!, uini, tspan, p)
sol = solve(prob, ROCK4(), abstol=tol, reltol=tol)
println("Number of function evaluations : ", sol.destats.nf)
println("Number of accepted steps :       ", sol.destats.naccept)
println("Number of rejected steps :       ", sol.destats.nreject)

err = abs.(sol[end] - uexa)

ps = plot(sol[end], size=(900,300), leg=false, xaxis = ("x"), margin=5mm, 
          title="Heat solution", titlefontsize=12, markershape=:xcross, markersize=2)
pe = plot(x, err, xaxis = ("x"), leg=false, size=(900,600), margin=5mm, 
          title="Error", titlefontsize=12, markershape=:xcross, markersize=2)
plot(ps, pe, layout=(2,1))

### Radau 5 integration

In [ ]:
xmin = -2.0
xmax =  2.0
nx = 400

tini = 0. 
tend = 0.1      
tspan = (tini, tend)

x = collect(range(xmin, xmax, length=nx))
freq = pi/((xmax-xmin)/2)
uini = cos.(freq*x)

uexa = exp.(-(freq^2)*(tend-tini))*uini

dx = (xmax-xmin)/(nx-1)

p = [nx, dx]

tol = 1e-6
prob = ODEProblem(heat1d!, uini, tspan, p)
sol = solve(prob, RadauIIA5(), abstol=tol, reltol=tol)
println("Number of function evaluations : ", sol.destats.nf)
println("Number of linear solves :        ", sol.destats.nsolve)
println("Number of accepted steps :       ", sol.destats.naccept)
println("Number of rejected steps :       ", sol.destats.nreject)

err = abs.(sol[end] - uexa)

ps = plot(sol[end], size=(900,300), leg=false, xaxis = ("x"), margin=5mm, 
          title="Heat solution", titlefontsize=12, markershape=:xcross, markersize=2)
pe = plot(x, err, xaxis = ("x"), leg=false, size=(900,600), margin=5mm, 
          title="Error", titlefontsize=12, markershape=:xcross, markersize=2)
plot(ps, pe, layout=(2,1))

### Rock4 integration (evolution of time step)

In [ ]:
xmin = -2.0
xmax =  2.0
nx = 400

tini = 0. 
tend = 0.5    
tspan = (tini, tend)

x = collect(range(xmin, xmax, length=nx))
freq = pi/((xmax-xmin)/2)
uini = cos.(freq*x)

uexa = exp.(-(freq^2)*(tend-tini))*uini

dx = (xmax-xmin)/(nx-1)

p = [nx, dx]

tol = 1e-6
prob = ODEProblem(heat1d!, uini, tspan, p)
sol = solve(prob, ROCK4(), abstol=tol, reltol=tol)
println("Number of function evaluations : ", sol.destats.nf)
println("Number of accepted steps :       ", sol.destats.naccept)
println("Number of rejected steps :       ", sol.destats.nreject)

dt = sol.t[2:end]-sol.t[1:end-1]

ps = plot(sol[end], size=(900,300), leg=false, xaxis = ("x"), margin=5mm, 
          title="Heat solution", titlefontsize=12, markershape=:xcross, markersize=2)
pe = plot(sol.t[1:end-1], dt, xaxis = ("t"), yaxis = ("dt"), size=(900,600), leg=false, margin=5mm, 
          title="Time step", titlefontsize=12, seriestype=:scatter, markershape=:xcross)
plot(ps, pe, layout=(2,1))